## Fast API
FastAPI is a modern and high-performance Python web framework used to build APIs quickly and efficiently. Designed with simplicity it allows developers to create RESTful APIs using Python's type hints which also enable automatic validation and error handling.

Python has support for optional "type hints" (also called "type annotations"). These "type hints" or annotations are a special syntax that allow declaring the type of a variable. By declaring types for your variables, editors and tools can give you better support.
```python
def get_full_name(first_name: str, last_name: str):
    full_name = first_name.title() + " " + last_name.title()
    return full_name


print(get_full_name("john", "doe"))
```
He we have declared that first_name and last_name as strings

In [8]:
first_name: str = "John" #first_name is declared as string and it can only be used as a string
fname: str | None = None #The value of fname can be str or None

### Installation
First we will install FastAPI using pip
```bash
$ pip install fastapi
```
And `uvicorn`: Uvicorn is a lightning-fast ASGI (Asynchronous Server Gateway Interface) web server implementation for Python. It is designed to efficiently run asynchronous web applications and APIs built with frameworks like FastAPI, Starlette, and Django.
```bash
$ pip install "uvicorn[standard]"
```

### Getting started
Below is a simple FastAPI app in main.py

In [ ]:
from typing import Union
from fastapi import FastAPI

app = FastAPI()

@app.get('/')
def read_root():
    return {"Hello":"World"}

@app.get('/items/{item_id}')
def read_item(item_id: int, q: Union[str, None] = None):
    return {"item_id":item_id, "q": q}

Run this app using Uvicorn
```bash
$ uvicorn main:app --reload
```
The app is live at `http://127.0.0.1:8000`

First we have imported FastAPI
```python
from fastapi import FastAPI
```
Then we have build an app
```python
app = FastAPI()
```
Next we implement a GET request api
```python
@app.get('/')
def read_root():
    return {"Hello":"World"}
```
When ever we are at root '/' the function below will run and will return a dictionary that will be displayed as JSON on the page.

Next we have implemented another api for end point /items/{item_id}, the item_id is a parameter that can be only int type as declared. When ever we are at the end point the function under it will run.
```python
@app.get('/items/{item_id}')
def read_item(item_id: int, q: Union[str, None] = None):
    return {"item_id":item_id, "q": q}
```
q: Union[str, None] = None, the variable q can be of datatype str or None and its default value is set to None. The function take two arguments - one is q and other is item_id which the parameter part of url /items/{item_id}.

### Pydantic models

Pydantic is a Python library to perform data validation.

You declare the "shape" of the data as classes with attributes.

And each attribute has a type.

Then you create an instance of that class with some values and it will validate the values, convert them to the appropriate type (if that's the case) and give you an object with all the data.

And you get all the editor support with that resulting object.

In [ ]:
from datetime import datetime

from pydantic import BaseModel


class User(BaseModel):
    id: int
    name: str = "John Doe"
    signup_ts: datetime | None = None
    friends: list[int] = []


external_data = {
    "id": "123",
    "signup_ts": "2017-06-01 12:22",
    "friends": [1, "2", b"3"],
}
user = User(**external_data)
print(user)
# > User id=123 name='John Doe' signup_ts=datetime.datetime(2017, 6, 1, 12, 22) friends=[1, 2, 3]
print(user.id)
# > 123

The attributes of model User like id which is not defined are required to be passed.

### Templates
You can use any template engine you want with FastAPI. A template engine is a software tool that takes a template file and combines it with data to generate output, most commonly HTML for web pages.

A common choice is Jinja2, the same one used by Flask and other tools.

```bash
$ pip install jinja2
```
First we will create a folder named 'static' that contains static files like HTML, CSS and JavaScript that runs at client side. And also a folder called templates that contains our templates.

In [ ]:
from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates

We have import some classes that are going to be used here
- FastAPI → Used to create your FastAPI app.
(Like app = FastAPI())

- Request → Represents the incoming HTTP request
(Used in routes to access request data)

- from fastapi.responses import HTMLResponse - FastAPI normally returns JSON, but if you want to return HTML, you need this.

- from fastapi.staticfiles import StaticFiles - Used to serve static files, such as: CSS, JavaScript, Images, Fonts

- from fastapi.templating import Jinja2Templates - This imports the template engine for HTML files. Jinja2Templates lets you:
    - Render .html files
    - Use variables inside HTML
    - Use template inheritance
    - Build dynamic pages

In [ ]:
app = FastAPI()
app.mount("/static", StaticFiles(directory="static"), name="static")
templates = Jinja2Templates(directory="templates")

- `app = FastAPI()` - This lets create a FastAPI app
- `app.mount("/static", StaticFiles(directory="static"), name="static")` - This line tells FastAPI: “Serve all files inside static/ folder through the URL /static.”  
    Meaning:
    -  Your CSS files → /static/style.css
    - Your JavaScript → /static/app.js
    - Images → /static/logo.png

    Breaking it down:
    - mount → Attach another app or file handler to a path
    - "/static" → The URL prefix
    - StaticFiles(directory="static") → Points to the static folder in your project
    - name="static" → A label to refer to this mount later

    It’s similar to Express.js: `app.use('/static', express.static('static'))`
- `templates = Jinja2Templates(directory="templates")` : This line tells FastAPI: “My HTML templates are inside the templates/ folder. Load them using Jinja2.”  
    Similar to express but express uses ***ejs*** as template engine
    ```js
    app.set("view engine", "ejs");
    app.set("views", "./views");
    ```

We will add html file in templates like index.html
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Index</title>
</head>
<body>
    I am the home page
</body>
</html>
```
and serve it on the root path

In [ ]:
@app.get('/',response_class=HTMLResponse)
async def read_home(request: Request):
    return templates.TemplateResponse('index.html',{'request': request})

- `@app.get('/', response_class=HTMLResponse)` : This is a route decorator that tells FastAPI:
    - @app.get('/') → Create a GET route for the home page at /.
    - response_class=HTMLResponse → This route should return HTML, not JSON. (FastAPI normally returns JSON by default.)

- `async def read_home(request: Request):` : This defines the request handler for the route.
    - async def → This is an asynchronous function (FastAPI supports async for speed).
    - read_home → The name of the function (your route handler).
    - request: Request → FastAPI automatically injects the incoming HTTP request object. Templates require request to work properly.

- `return templates.TemplateResponse('index.html', {'request': request})` : This line tells FastAPI to render an HTML template.
    - templates.TemplateResponse → Uses Jinja2 to load and render HTML files.
    - 'index.html' → FastAPI will look for this file inside your templates/ folder.
    - {'request': request} → You must pass the request into the template. Jinja2 needs this for internal rendering logic.

Now we will create a dynamic page where a value changes dynamically the value id in page.html
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Page: {{id}}</title>
</head>
<body>
    This is page no: {{id}}
</body>
</html>
```

In [ ]:
@app.get("/page/{id}", response_class=HTMLResponse)
async def read_page(request: Request, id: str):
    return templates.TemplateResponse(
        request=request, name="page.html", context={"id": id}
    )

We can route to pages like /page/1, page/2, .... page/100 etc  

- `@app.get("/page/{id}", response_class=HTMLResponse)` : This is a route decorator.
    - `@app.get("/page/{id}")` : Creates a GET route like:
        - /page/10
        - /page/abc
        - /page/xyz123
    - {id} is a path parameter, meaning whatever text appears in that position becomes a variable called id.

- `async def read_page(request: Request, id: str):` : This defines the route handler function.
    - `id: str` : This is the dynamic value from the URL.
        Example: If user visits /page/25, then: id = "25"

- `return templates.TemplateResponse` : Starts the return statement to send an HTML template as response.
    - `request=request`: This passes the request object into the template. Templates require this for internal usage. But note: In other calls, you'd normally pass it inside a context dictionary like: `{"request": request}`. BUT here, FastAPI also allows passing it as a separate argument.
    - `name="page.html"`: Tells FastAPI to render the file: templates/page.html
    - `context={"id": id}`: This is the data you pass to the HTML template. Inside page.html you can access it like: `<p>The ID is: {{ id }}</p>`

### Pymongo
